# <h1 align=center> **MACHINE LEARNING** </h1>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns
import warnings
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split
import pyarrow.parquet as pq
from surprise import SVD
from surprise import accuracy
import pickle

In [8]:
# Suprimir todos los warnings
warnings.filterwarnings("ignore")

In [4]:
games=pd.read_parquet("games.parquet")
reviews=pd.read_parquet("reviews.parquet")

Merge de los DataFrames reviews con games unidos por item_id

In [5]:
merged_df = pd.merge(reviews[['item_id', 'user_id', 'sentiment_analysis',"recommend"]], games[['app_name',"item_id"]], left_on='item_id', right_on="item_id")

In [6]:
merged_df

,item_id,user_id,sentiment_analysis,recommend,app_name
0,1250,76561197970982479,2,True,Killing Floor
1,1250,death-hunter,2,True,Killing Floor
2,1250,DJKamBer,0,True,Killing Floor
3,1250,diego9031,1,True,Killing Floor
4,1250,76561198081962345,1,True,Killing Floor
...,...,...,...,...,...
44734,367780,laislabonita75,2,True,Aero's Quest
44735,367780,evilindiegaming,2,True,Aero's Quest
44736,305920,laislabonita75,0,False,Another Perspective
44737,306040,Gamer0009,2,True,The Howler


Reemplazamos por 1 y 0 la columna recommend

In [7]:
merged_df['recommend'] = merged_df['recommend'].replace({True: 1, False: 0})

Hacemos una columna nueva llamada rating 

In [11]:
for index, row in merged_df.iterrows():
    if (row[2] == 2) and (row[3] == 1):
        merged_df.at[index, 'rating'] = 5
    elif (row[2] == 2) and (row[3] == 0):
        merged_df.at[index, 'rating'] = 4
    elif (row[2] == 1) and (row[3] == 1):
        merged_df.at[index, 'rating'] = 3
    elif (row[2] == 1) and (row[3] == 0):
        merged_df.at[index, 'rating'] = 2
    elif (row[2] == 0) and (row[3] == 1):
        merged_df.at[index, 'rating'] = 1
    elif (row[2] == 0) and (row[3] == 0):
        merged_df.at[index, 'rating'] = 0

In [12]:
merged_df

,item_id,user_id,sentiment_analysis,recommend,app_name,rating
0,1250,76561197970982479,2,1,Killing Floor,5.0
1,1250,death-hunter,2,1,Killing Floor,5.0
2,1250,DJKamBer,0,1,Killing Floor,1.0
3,1250,diego9031,1,1,Killing Floor,3.0
4,1250,76561198081962345,1,1,Killing Floor,3.0
...,...,...,...,...,...,...
44734,367780,laislabonita75,2,1,Aero's Quest,5.0
44735,367780,evilindiegaming,2,1,Aero's Quest,5.0
44736,305920,laislabonita75,0,0,Another Perspective,0.0
44737,306040,Gamer0009,2,1,The Howler,5.0


Guardamos el Dataset en un nuevo archivo .parquet

In [13]:
merged_df.to_parquet('modelo.parquet')

Leemos el archivo .parquet y lo cargarlo en un DataFrame de pandas

In [14]:
new_df = pd.read_parquet('modelo.parquet')

In [15]:
new_df

,item_id,user_id,sentiment_analysis,recommend,app_name,rating
0,1250,76561197970982479,2,1,Killing Floor,5.0
1,1250,death-hunter,2,1,Killing Floor,5.0
2,1250,DJKamBer,0,1,Killing Floor,1.0
3,1250,diego9031,1,1,Killing Floor,3.0
4,1250,76561198081962345,1,1,Killing Floor,3.0
...,...,...,...,...,...,...
44734,367780,laislabonita75,2,1,Aero's Quest,5.0
44735,367780,evilindiegaming,2,1,Aero's Quest,5.0
44736,305920,laislabonita75,0,0,Another Perspective,0.0
44737,306040,Gamer0009,2,1,The Howler,5.0


Adaptar rating_scale a a escala de datos

In [17]:
reader = Reader(rating_scale=(0, 5))  

Cargamos los datos de los usuarios, juegos y calificaciones en un objeto Dataset de Surprise

In [18]:
data = Dataset.load_from_df(new_df[['user_id', 'app_name', 'rating']], reader)

Se divide el conjunto de datos en entrenamiento y prueba

In [19]:
trainset, testset = train_test_split(data, test_size=0.2,random_state=42)  

Optimizacion de hiperparaétros con GridSeachCV

In [20]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_factors': [5,50,100],'n_epochs': [5, 10,20], 'lr_all': [0.001, 0.002, 0.005],
              'reg_all': [0.002, 0.02, 0.2]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=5, n_jobs = -1)
gs.fit(data)

Observamos el RMSE y los hiperparamétros para nuestro modelo 

In [21]:
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

1.5482129323665461
{'n_factors': 100, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.2}


Seleccionamos el algoritmo SVD con sus hiperparamétros  y entrenamos el modelo

In [24]:
model = SVD(n_factors=100, n_epochs=20, lr_all=0.005, reg_all=0.2)
model.fit(trainset)

Guardamos el modelo en un archivo .pkl

In [25]:
with open('modelo.pkl', 'wb') as archivo:
    pickle.dump(model, archivo)

Abrimos el archivo .pkl y lo guardamos en una variable

In [26]:
with open ('modelo.pkl', 'rb') as archivo:
    modelo = pickle.load(archivo)

In [27]:
print(modelo)

Realizamos la función de recomendación 

In [38]:
def recomendacion_usuario( id_usuario ):
    lista=[]
    if id_usuario not in new_df['user_id'].unique():
        return {'error': 'El usuario especificado no existe.'}
    
    #Crear una lista de juegos valorados por el usuario
    juegos_valorados = new_df[new_df['user_id'] == id_usuario]['app_name'].unique()

    #Crear una lista de todos los juegos disponibles
    todos_los_juegos = new_df['app_name'].unique()

    #Crear una lista de juegos no valorados por el usuario específico
    juegos_no_valorados = list(set(todos_los_juegos) - set(juegos_valorados))

    #Generar predicciones para los juegos no valorados por el usuario
    predicciones = [modelo.predict(id_usuario, juego) for juego in juegos_no_valorados]

    #Ordenar las predicciones en base a la valoración y obtener los juegos recomendados
    recomendaciones = sorted(predicciones, key=lambda x: x.est, reverse=True)[:5]  # Obtener las 5 mejores recomendaciones

  # Almacenar los nombres de los juegos recomendados en una lista
    juegos_recomendados = [recomendacion.iid for recomendacion in recomendaciones]
    
    # Crear un diccionario con los nombres de los juegos recomendados
    recomendaciones_dict = {
        'Juego 1': juegos_recomendados[0],
        'Juego 2': juegos_recomendados[1],
        'Juego 3': juegos_recomendados[2],
        'Juego 4': juegos_recomendados[3],
        'Juego 5': juegos_recomendados[4]
    }
    
    # Devolver el diccionario con los nombres de los juegos recomendados
    return recomendaciones_dict
    

In [39]:
recomendacion_usuario('inorisanbaka')

{'Juego 1': 'Bastion',
 'Juego 2': 'Remember Me',
 'Juego 3': 'Dragon Nest',
 'Juego 4': 'SMITE®',
 'Juego 5': 'Dust: An Elysian Tail'}

In [40]:
recomendacion_usuario("diego9031")

{'Juego 1': 'Bastion',
 'Juego 2': 'SMITE®',
 'Juego 3': 'Supreme Commander: Forged Alliance',
 'Juego 4': 'Rocksmith® 2014 Edition - Remastered',
 'Juego 5': 'Psychonauts'}

In [41]:
recomendacion_usuario("")

{'error': 'El usuario especificado no existe.'}

In [42]:
recomendacion_usuario("Gamer0009")

{'Juego 1': 'RIFT',
 'Juego 2': 'SMITE®',
 'Juego 3': 'Dragon Nest',
 'Juego 4': 'Bastion',
 'Juego 5': 'Halo: Spartan Assault'}